In [ ]:
import cv2 
import controller
from cvzone.HandTrackingModule import HandDetector
import pyfirmata
import speech_recognition as sr
import wave
from pydub import AudioSegment
import re
from faster_whisper import WhisperModel


In [ ]:

comport='COM12'#chech the COM port number u got to connect ardunio in your system

board=pyfirmata.Arduino(comport)


led_1=board.get_pin('d:8:o')
led_2=board.get_pin('d:9:o')
led_3=board.get_pin('d:10:o')


def led(fingerUp):
    if fingerUp==[1,1,0,0,0]:
        led_1.write(1)

    elif fingerUp==[0,0,0,0,1]:
        led_1.write(0)

    elif fingerUp==[1,1,1,0,0]: 
        led_2.write(1)

    elif fingerUp==[0,1,0,0,1]:
        led_2.write(0)

    elif fingerUp==[1,1,1,1,0]:
        led_3.write(1)

    elif fingerUp==[1,1,0,0,1]:
        led_3.write(0)

In [ ]:
detector=HandDetector(detectionCon=0.8,maxHands=1)
video=cv2.VideoCapture(0)
 
while True:
    ret,frame=video.read()
    frame=cv2.flip(frame,1)
    hands,img=detector.findHands(frame)
    if hands:
        lmList=hands[0]
        #print(hands)
        fingerUp=detector.fingersUp(lmList)
        led(fingerUp)
        if fingerUp==[0,0,0,0,0]:
            cv2.putText(frame,'Finger count:0',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        elif fingerUp==[1,1,0,0,0]:
            cv2.putText(frame,'on first light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)    
        elif fingerUp==[0,0,0,0,1]:
            cv2.putText(frame,'off first light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        elif fingerUp==[1,1,1,0,0]:
            cv2.putText(frame,'on second light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        elif fingerUp==[0,1,0,0,1]:
            cv2.putText(frame,'off second light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        elif fingerUp==[1,1,1,1,0]:
            cv2.putText(frame,'on third light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA) 
        elif fingerUp==[1,1,0,0,1]:
            cv2.putText(frame,'off third light',(20,460),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
    cv2.imshow("frame",frame)
    k=cv2.waitKey(1)
    if k==ord("k"):
        break
video.release()
cv2.destroyAllWindows()

In [ ]:


model_size = "large-v3"

model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [ ]:


def control():
    while True:
# Capture audio
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            print("Speak now...")
            audio = recognizer.listen(source)
        
        # Save temporary audio (e.g., .wav)
        with wave.open("instructions.wav", "wb") as f:
            f.setframerate(44100)  # Adjust framerate as needed
            f.setnchannels(1)  # Adjust channels as needed
            f.setsampwidth(2)  # Adjust sample width as needed
            f.writeframes(audio.frame_data)
        
        # Convert .wav to .mp3
        sound = AudioSegment.from_file("instructions.wav")
        sound.export("instructions.mp3", format="mp3")
        
        segments, info = model.transcribe("instructions.mp3", beam_size=5, language="en", condition_on_previous_text=False)
        
        for segment in segments:
            #print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
            print(segment.text)
        
        def remove_special_chars(text):
         # Combine steps: lowercase and remove special characters
          words = re.sub(r"[^\w\s]", "", text.lower()).split()
          return words
        
        # Example usage (assuming segment is an object with a 'text' attribute)
        words = remove_special_chars(segment.text)
        
        print(words)
        for i in range(len(words)):
            if (words[i]=="on"):
                    if(words[i+2]=="light"):
                        x=[1,1,0,0,0]
                        led(x)
                    if(words[i+2]=="fan"):
                        x=[1,1,1,0,0]
                        led(x)
                    if(words[i+2]=="tv"):
                        x=[1,1,1,1,0]
                        led(x)
                    
    
            if (words[i]=="of" or words[i]=="off" ):
                    if(words[i+2]=="tv"):
                        x=[1,1,0,0,1]
                        led(x)
                    if(words[i+2]=="fan"):
                        x=[0,1,0,0,1]
                        led(x)
                    if(words[i+2]=="light"):
                        x=[0,0,0,0,1]
                        led(x)
                    if(words[i+2]=="system"):
                        x=[1,1,0,0,1]
                        led(x)
                        x=[0,1,0,0,1]
                        led(x)
                        x=[0,0,0,0,1]
                        led(x)
                        return()